# Some clustering of the NAS and original patterns by vector size to try and establish a relationship between labels

Load libraries and dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.utils import to_categorical
import seaborn as sns
dataset = pd.read_csv('patterns.csv',sep=';')
dataset_NAS = pd.read_csv('NAS-clean.csv',sep=';')

Using TensorFlow backend.
/home/ssleder/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ssleder/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ssleder/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ssleder/.local/lib/python3.6/site-packages/tensorflow/py

Shuffle data, necessary to mix the rows in the dataset in order to have training and testing data with all the cases.
If the data is not shuffled, the cur for the test7train pair will be done at exacty the x-th position in the dataset.

In [2]:
from sklearn.utils import shuffle

# drop rows with NAN values
dataset = dataset.dropna()
dataset_NAS = dataset_NAS.dropna()

# shuffle the datasets
dataset = shuffle(dataset)
dataset_NAS = shuffle(dataset_NAS)

In [3]:
sorted(dataset['size vector'].unique())

[1000,
 2000,
 3000,
 4000,
 5000,
 6000,
 7000,
 8000,
 9000,
 10000,
 20000,
 30000,
 40000,
 50000,
 60000,
 70000,
 80000,
 90000,
 100000,
 200000,
 300000,
 400000,
 500000,
 600000,
 700000,
 800000,
 900000,
 1000000,
 2000000,
 3000000,
 5000000,
 7000000,
 9000000,
 20000000,
 40000000,
 60000000,
 80000000,
 100000000,
 120000000,
 140000000,
 160000000,
 180000000,
 200000000]

Remove extra columns and hardware performance counters discarded with PCA

In [4]:
from sklearn.cluster import KMeans

def find_num_clusters(vector_size, max_clusters=15):
    wcss = []
    for i in range(1, max_clusters):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=100, n_init=10, n_jobs=-1)
        kmeans.fit(vector_size)
        wcss.append(kmeans.inertia_)
    plt.plot(range(1, max_clusters), wcss)
    plt.title('Elbow Method')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.show()

In [8]:
def erase_columns(dataset_in):
    del dataset_in['id']
    del dataset_in['size vector']
    del dataset_in['comp opt']
    del dataset_in['threads']
    del dataset_in['affinity']
    #del dataset_in['label']
    dataset = shuffle(dataset_in)
    dataset_in = dataset_in.reset_index(drop=True)
    #dataset_in.info()

Empty DataFrame
Columns: [id, size vector, comp opt, label, threads, PAPI_L1_DCM, PAPI_L3_TCM, PAPI_TOT_INS, PAPI_REF_CYC, PAPI_L2_LDM, PAPI_SR_INS, PAPI_BR_UCN, PAPI_BR_CN, PAPI_BR_TKN, PAPI_BR_MSP, PAPI_FP_OPS, PAPI_L1_LDM, PAPI_L1_STM, PAPI_SP_OPS, PAPI_LD_INS, PAPI_L2_STM, PAPI_TLB_DM, PAPI_L2_DCH, affinity]
Index: []

[0 rows x 24 columns]


In [16]:
from sklearn.preprocessing import Normalizer

vector_sizes = {}

for s in sorted(dataset['size vector'].unique()):
    # get the vector size and append the NAS dataset
    tmp_vect = dataset[dataset['size vector'] == s]#.append(dataset_NAS)
        
    # erase redundant columns
    erase_columns(tmp_vect)
    
    # perform normalization on the numberical columns (not labels)
    for i in range(1, 19):
        scaler = Normalizer()
        scaler.fit(tmp_vect[tmp_vect.columns[i]].values.reshape(-1, 1))
        scaled = scaler.transform(tmp_vect[tmp_vect.columns[i]].values.reshape(-1, 1))
        
        tmp_vect[tmp_vect.columns[i]] = scaled
    '''    
    tmp_vect[tmp_vect.columns[1:19]] = \
            tmp_vect[tmp_vect.columns[1:19]].apply(
                    lambda x: (x - x.min()) / (x.max() - x.min()))
    '''
    vector_sizes[s] = tmp_vect
    print(vector_sizes[s].shape)

/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

(36000, 19)
(36000, 19)
(36000, 19)
(36000, 19)


/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

(36000, 19)
(36000, 19)
(36000, 19)
(36000, 19)


/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

...

/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

(36000, 19)
(36000, 19)
(36000, 19)
(36000, 19)


/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

(36000, 19)
(36000, 19)
(36000, 19)
(36000, 19)
(36000, 19)
(36000, 19)
(36000, 19)


/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ssleder/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [7]:
for key_vector_size, vector_size_set in vector_sizes.items():
    print(key_vector_size, vector_size_set.shape)
    print(vector_size_set[vector_size_set.isnull().any(axis=1)])
    #print(vector_size_set['size vector'].unique())
    #find_num_clusters(vector_size_set.iloc[:,1:19])

1000 (36000, 19)
Empty DataFrame
Columns: [label, PAPI_L1_DCM, PAPI_L3_TCM, PAPI_TOT_INS, PAPI_REF_CYC, PAPI_L2_LDM, PAPI_SR_INS, PAPI_BR_UCN, PAPI_BR_CN, PAPI_BR_TKN, PAPI_BR_MSP, PAPI_FP_OPS, PAPI_L1_LDM, PAPI_L1_STM, PAPI_SP_OPS, PAPI_LD_INS, PAPI_L2_STM, PAPI_TLB_DM, PAPI_L2_DCH]
Index: []
2000 (36000, 19)
Empty DataFrame
Columns: [label, PAPI_L1_DCM, PAPI_L3_TCM, PAPI_TOT_INS, PAPI_REF_CYC, PAPI_L2_LDM, PAPI_SR_INS, PAPI_BR_UCN, PAPI_BR_CN, PAPI_BR_TKN, PAPI_BR_MSP, PAPI_FP_OPS, PAPI_L1_LDM, PAPI_L1_STM, PAPI_SP_OPS, PAPI_LD_INS, PAPI_L2_STM, PAPI_TLB_DM, PAPI_L2_DCH]
Index: []
3000 (36000, 19)
Empty DataFrame
Columns: [label, PAPI_L1_DCM, PAPI_L3_TCM, PAPI_TOT_INS, PAPI_REF_CYC, PAPI_L2_LDM, PAPI_SR_INS, PAPI_BR_UCN, PAPI_BR_CN, PAPI_BR_TKN, PAPI_BR_MSP, PAPI_FP_OPS, PAPI_L1_LDM, PAPI_L1_STM, PAPI_SP_OPS, PAPI_LD_INS, PAPI_L2_STM, PAPI_TLB_DM, PAPI_L2_DCH]
Index: []
4000 (36000, 19)
Empty DataFrame
Columns: [label, PAPI_L1_DCM, PAPI_L3_TCM, PAPI_TOT_INS, PAPI_REF_CYC, PAPI_L2_LDM

Empty DataFrame
Columns: [label, PAPI_L1_DCM, PAPI_L3_TCM, PAPI_TOT_INS, PAPI_REF_CYC, PAPI_L2_LDM, PAPI_SR_INS, PAPI_BR_UCN, PAPI_BR_CN, PAPI_BR_TKN, PAPI_BR_MSP, PAPI_FP_OPS, PAPI_L1_LDM, PAPI_L1_STM, PAPI_SP_OPS, PAPI_LD_INS, PAPI_L2_STM, PAPI_TLB_DM, PAPI_L2_DCH]
Index: []
3000000 (36000, 19)
Empty DataFrame
Columns: [label, PAPI_L1_DCM, PAPI_L3_TCM, PAPI_TOT_INS, PAPI_REF_CYC, PAPI_L2_LDM, PAPI_SR_INS, PAPI_BR_UCN, PAPI_BR_CN, PAPI_BR_TKN, PAPI_BR_MSP, PAPI_FP_OPS, PAPI_L1_LDM, PAPI_L1_STM, PAPI_SP_OPS, PAPI_LD_INS, PAPI_L2_STM, PAPI_TLB_DM, PAPI_L2_DCH]
Index: []
5000000 (36000, 19)
Empty DataFrame
Columns: [label, PAPI_L1_DCM, PAPI_L3_TCM, PAPI_TOT_INS, PAPI_REF_CYC, PAPI_L2_LDM, PAPI_SR_INS, PAPI_BR_UCN, PAPI_BR_CN, PAPI_BR_TKN, PAPI_BR_MSP, PAPI_FP_OPS, PAPI_L1_LDM, PAPI_L1_STM, PAPI_SP_OPS, PAPI_LD_INS, PAPI_L2_STM, PAPI_TLB_DM, PAPI_L2_DCH]
Index: []
7000000 (36000, 19)
Empty DataFrame
Columns: [label, PAPI_L1_DCM, PAPI_L3_TCM, PAPI_TOT_INS, PAPI_REF_CYC, PAPI_L2_LDM, PAPI_S